# PyTorch Tutorial
KAIST, October 2018

By Dongkyum Kim

Useful tutorials of python
- https://www.scipy-lectures.org/
- https://docs.python.org/ko/3/tutorial/index.html

In [2]:
from __future__ import print_function
import numpy as np
import torch 
import matplotlib.pyplot as plt

# Advanced Topcis

## Define-by-run Paradigm

The torch autograd package provides automatic differentiation for all operations on Tensors.

PyTorch's autograd is a reverse mode automatic differentiation system.

Backprop is defined by how your code is run, and that every single iteration can be different.

Other frameworks that adopt a similar approach :

1. Chainer - https://github.com/chainer/chainer
2. DyNet - https://github.com/clab/dynet
3. Tensorflow Eager - https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html

### How autograd encodes execution history


Conceptually, autograd maintains a graph that records all of the operations performed on variables as you execute your operations. This results in a directed acyclic graph whose leaves are the input tensors and roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

![caption](https://raw.githubusercontent.com/pytorch/pytorch/master/docs/source/_static/img/dynamic_graph.gif)

GIF source: https://github.com/pytorch/pytorch

Internally, autograd represents this graph as a graph of Function objects (really expressions), which can be `apply()` ed to compute the result of evaluating the graph. When computing the forwards pass, autograd simultaneously performs the requested computations and builds up a graph representing the function that computes the gradient (the `.grad_fn` attribute of each Variable is an entry point into this graph). When the forwards pass is completed, we evaluate this graph in the backwards pass to compute the gradients.

In [3]:
x = torch.randn(5, 3, requires_grad=True)
y = torch.randn(3, 5)
z = torch.mm(x, y)
print(z.grad_fn)

An important thing to note is that the graph is recreated from scratch at every iteration, and this is exactly what allows for using arbitrary Python control flow statements, that can change the overall shape and size of the graph at every iteration. You don’t have to encode all possible paths before you launch the training - what you run is what you differentiate.

## Getting gradients : `backward()` & `torch.autograd.grad`

In [4]:
x = torch.randn(5, 3, requires_grad=True)
y = torch.randn(5, 3, requires_grad=True)
z = x ** 2 + 3 * y
z.backward(gradient=torch.ones(5, 3))
print(x)

tensor([[-4.8153e-01, -1.3644e+00,  1.5768e-01],
        [ 1.9485e-01, -5.8349e-01,  3.2768e-01],
        [-1.7590e+00, -8.1949e-04, -1.5361e+00],
        [ 1.5284e+00,  1.1169e+00,  1.6429e+00],
        [ 4.4778e-01, -2.1676e-01,  9.8591e-01]], requires_grad=True)


In [5]:
x.grad

tensor([[-9.6305e-01, -2.7289e+00,  3.1537e-01],
        [ 3.8971e-01, -1.1670e+00,  6.5536e-01],
        [-3.5181e+00, -1.6390e-03, -3.0722e+00],
        [ 3.0567e+00,  2.2337e+00,  3.2858e+00],
        [ 8.9557e-01, -4.3352e-01,  1.9718e+00]])

In [6]:
y.grad

tensor([[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]])

In [7]:
# eq computes element-wise equality
torch.eq(x.grad, 2 * x)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True]])

In [8]:
x = torch.randn(5, 3, requires_grad=True)
y = torch.randn(5, 3, requires_grad=True)
z = x ** 2 + 3 * y
dz_dx = torch.autograd.grad(z, x, grad_outputs=torch.ones(5, 3))
dz_dy = torch.autograd.grad(z, y, grad_outputs=torch.ones(5, 3))

## Define-by-run example

### Common Variable definition

In [9]:
x = torch.randn(5, 3, requires_grad=True)
w = torch.randn(3, 10, requires_grad=True)
b = torch.randn(10, requires_grad=True)

In [10]:
print(x.data)
print(w.data)
print(b.data)

tensor([[ 0.2177, -2.6580,  1.0147],
        [-0.6363, -0.6095,  1.7360],
        [-0.3301, -1.1459,  0.1440],
        [ 1.2289,  1.0900, -0.4834],
        [ 1.5760, -0.5638, -0.1654]])
tensor([[ 1.2075,  0.0066,  0.8835, -0.1077,  1.9702, -0.9342,  1.1622, -0.3672,
         -0.8283, -1.8957],
        [ 1.9502,  1.4504,  0.6729,  1.2803, -0.9831,  1.2466, -0.5352, -0.4427,
         -0.5211,  0.1783],
        [ 1.7587,  0.4763,  0.1377, -0.1767, -0.4174, -2.3581, -0.0181, -0.5909,
          0.1494, -0.0503]])
tensor([-1.2090,  2.4478,  0.6289, -0.4855, -0.1159, -0.5819,  0.0804,  0.0051,
         0.8396,  0.3468])


### Graph 1 : `wx + b`

In [11]:
o = torch.matmul(x, w) + b
print(o)
do_dinputs_1 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

tensor([[-4.3450, -0.9225, -0.8276, -4.0915,  2.5027, -6.4917,  1.7377,  0.5024,
          2.1961, -0.5907],
        [-0.1129,  2.3864, -0.1043, -1.5042, -1.4949, -4.8409, -0.3643, -0.5172,
          1.9437,  1.3572],
        [-3.5890,  0.8522, -0.4139, -1.9425,  0.3003, -2.0415,  0.3076,  0.5486,
          1.7316,  0.7611],
        [ 1.5506,  3.8066,  2.3816,  0.8632,  1.4354,  0.7687,  0.9340, -0.6430,
         -0.8186, -1.7643],
        [-0.6963,  1.5617,  1.6192, -1.3479,  3.6125, -2.3671,  2.2168, -0.2262,
         -0.1967, -2.7331]], grad_fn=<AddBackward0>)


In [12]:
print('Gradients of o w.r.t inputs in Graph 1')
print('do/dx : \n\n %s ' % (do_dinputs_1[0]))
print('do/dw : \n\n %s ' % (do_dinputs_1[1]))
print('do/db : \n\n %s ' % (do_dinputs_1[2]))

Gradients of o w.r.t inputs in Graph 1
do/dx : 

 tensor([[ 1.0969,  4.2964, -1.0893],
        [ 1.0969,  4.2964, -1.0893],
        [ 1.0969,  4.2964, -1.0893],
        [ 1.0969,  4.2964, -1.0893],
        [ 1.0969,  4.2964, -1.0893]]) 
do/dw : 

 tensor([[ 2.0563,  2.0563,  2.0563,  2.0563,  2.0563,  2.0563,  2.0563,  2.0563,
          2.0563,  2.0563],
        [-3.8872, -3.8872, -3.8872, -3.8872, -3.8872, -3.8872, -3.8872, -3.8872,
         -3.8872, -3.8872],
        [ 2.2459,  2.2459,  2.2459,  2.2459,  2.2459,  2.2459,  2.2459,  2.2459,
          2.2459,  2.2459]]) 
do/db : 

 tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]) 


### Graph 2 : wx / b

In [13]:
o = torch.matmul(x, w) / b
do_dinputs_2 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [14]:
print('Gradients of o w.r.t inputs in Graph 2')
print('do/dx : \n %s ' % (do_dinputs_2[0]))
print('do/dw : \n %s ' % (do_dinputs_2[1]))
print('do/db : \n %s ' % (do_dinputs_2[2]))

Gradients of o w.r.t inputs in Graph 2
do/dx : 
 tensor([[ -78.0972,  -89.0173, -108.0126],
        [ -78.0972,  -89.0173, -108.0126],
        [ -78.0972,  -89.0173, -108.0126],
        [ -78.0972,  -89.0173, -108.0126],
        [ -78.0972,  -89.0173, -108.0126]]) 
do/dw : 
 tensor([[  -1.7009,    0.8401,    3.2694,   -4.2352,  -17.7416,   -3.5337,
           25.5621,  399.4747,    2.4492,    5.9285],
        [   3.2153,   -1.5881,   -6.1805,    8.0062,   33.5385,    6.6800,
          -48.3222, -755.1607,   -4.6299,  -11.2072],
        [  -1.8577,    0.9175,    3.5708,   -4.6257,  -19.3772,   -3.8594,
           27.9186,  436.3015,    2.6750,    6.4751]]) 
do/db : 
 tensor([ 7.8524e-01,  7.6015e-01,  1.2382e+00,  2.3736e+01, -5.1628e+02,
         3.5622e+01, -6.8451e+02,  1.3629e+04, -9.3367e-01,  3.9101e+01]) 


## Gradient buffers: `.backward()` and `retain_graph=True`

1. Calling `.backward()` clears the current computation graph.
2. Once `.backward()` is called, intermediate variables used in the construction of the graph are removed.
2. This is used implicitly to let PyTorch know when a new graph is to be built for a new minibatch. This is built around the forward and backward pass paradigm.
3. To retain the graph after the backward pass use `loss.backward(retain_graph=True)`. This lets you re-use intermediate variables to potentially compute a secondary loss after the initial gradients are computed. This is useful to implement things like the gradient penalty in WGANs (https://arxiv.org/abs/1704.00028)

In [15]:
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10))

### Call backward again -> <font color='red'>This fails</font>

In [16]:
o = o ** 3
o.backward(torch.ones(5, 10))

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

###  But with `retain_graph=True`

In [17]:
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10), retain_graph=True)

In [18]:
# o = o ** 3
o.backward(torch.ones(5, 10), retain_graph=True)
print(x.grad)

tensor([[ 3.2906, 12.8893, -3.2678],
        [ 3.2906, 12.8893, -3.2678],
        [ 3.2906, 12.8893, -3.2678],
        [ 3.2906, 12.8893, -3.2678],
        [ 3.2906, 12.8893, -3.2678]])


## <font color='red'>WARNING:</font> Calling `.backward()` multiple times will accumulate gradients into `.grad` and NOT overwrite them.

## Excluding subgraphs from backward: requires_grad=False & .detach

### `requires_grad=False`

1. If there’s a single input to an operation that requires gradient, its output will also require gradient.

2. Conversely, if all inputs don’t require gradient, the output won’t require it.

3. Backward computation is never performed in the subgraphs, where all tensors didn’t require gradients.

4. This is potentially useful when you have part of a network that is pretrained and not fine-tuned, for example word embeddings or a pretrained imagenet model.

In [19]:
x = torch.randn(3, 5, requires_grad=False)
y = torch.randn(3, 5, requires_grad=False)
z = torch.randn(3, 5, requires_grad=True)

In [20]:
o = x + y
print(' o = x + y requires grad ? : %s ' % (o.requires_grad))
o = x + y + z
print(' o = x + y + z requires grad ? : %s ' % (o.requires_grad))

 o = x + y requires grad ? : False 
 o = x + y + z requires grad ? : True 


### `.detach()`

1. It is possible to detach variables from the graph by calling `.detach()`. 
2. This could lead to disconnected graphs. In which case PyTorch will only backpropagate gradients until the point of disconnection.

In [21]:
x = torch.randn(3, 5, requires_grad=True)
y = torch.randn(3, 5, requires_grad=True)
z = torch.randn(3, 5, requires_grad=True)

![caption](images/detach.png)

In [22]:
m1 = x + y
m2 = z ** 2
m1 = m1.detach()
m3 = m1 + m2
m3.backward(torch.ones(3, 5))

In [23]:
print('dm3/dx \n\n %s ' % (x.grad))
print('\ndm3/dy \n\n %s ' % (y.grad))
print('\ndm3/dz \n\n %s ' % (z.grad))

dm3/dx 

 None 

dm3/dy 

 None 

dm3/dz 

 tensor([[-1.6327,  2.4392, -2.1404, -0.6010,  1.7375],
        [ 1.0010,  4.8004,  0.3130, -1.2567, -1.4230],
        [ 4.1334,  1.3819,  0.5394, -3.1082,  1.4671]]) 


In [24]:
2*z

tensor([[-1.6327,  2.4392, -2.1404, -0.6010,  1.7375],
        [ 1.0010,  4.8004,  0.3130, -1.2567, -1.4230],
        [ 4.1334,  1.3819,  0.5394, -3.1082,  1.4671]], grad_fn=<MulBackward0>)

## Gradients w.r.t intermediate variables in the graph

1. By default, PyTorch all gradient computations w.r.t intermediate nodes in the graph are ad-hoc.

2. This is in the interest of saving memory.

3. To compute gradients w.r.t intermediate variables, use `.retain_grad()` or explicitly compute gradients using `torch.autograd.grad`

4. `.retain_grad()` populates the `.grad` attribute of the Variable while `torch.autograd.grad` returns a Variable that contains the gradients.

In [25]:
x = torch.randn(3, 5, requires_grad=True)
y = torch.randn(3, 5, requires_grad=True)
z = torch.randn(3, 5, requires_grad=True)

In [26]:
m1 = x + y
m2 = z ** 2
m1.retain_grad()
m2.retain_grad()
m3 = m1 * m2
m3.backward(torch.ones(3, 5), retain_graph=True)
print(m3)

tensor([[ 0.3754,  1.0937, -0.2501,  6.1322,  0.0873],
        [ 1.2120, -0.6478,  1.8503,  0.1360,  0.0069],
        [ 2.4660,  4.4501, -0.0614,  0.0776,  0.1646]], grad_fn=<MulBackward0>)


In [27]:
print('dm3/dm1 \n\n %s ' % (m1.grad))
print('dm3/dm2 \n\n %s ' % (m2.grad))

dm3/dm1 

 tensor([[0.6212, 0.5811, 0.2544, 4.1275, 0.8646],
        [1.0281, 0.2820, 3.6466, 0.5199, 0.0090],
        [1.5856, 3.1191, 0.2506, 0.2187, 0.9117]]) 
dm3/dm2 

 tensor([[ 0.6044,  1.8822, -0.9831,  1.4857,  0.1010],
        [ 1.1788, -2.2968,  0.5074,  0.2616,  0.7638],
        [ 1.5553,  1.4267, -0.2450,  0.3549,  0.1806]]) 


In [28]:
torch.autograd.grad(m3, [m1, m2], grad_outputs=torch.ones_like(m3))

(tensor([[0.6212, 0.5811, 0.2544, 4.1275, 0.8646],
         [1.0281, 0.2820, 3.6466, 0.5199, 0.0090],
         [1.5856, 3.1191, 0.2506, 0.2187, 0.9117]]),
 tensor([[ 0.6044,  1.8822, -0.9831,  1.4857,  0.1010],
         [ 1.1788, -2.2968,  0.5074,  0.2616,  0.7638],
         [ 1.5553,  1.4267, -0.2450,  0.3549,  0.1806]]))

### In place operations on variables in a graph

source: http://pytorch.org/docs/master/notes/autograd.html

In place operations are suffixed by `_` ex: `log_`, `uniform_` etc.

1. Supporting in-place operations in autograd is difficult and PyTorch discourages their use in most cases.

2. Autograd’s aggressive buffer freeing and reuse makes it very efficient and there are very few occasions when in-place operations actually lower memory usage by any significant amount. Unless you’re operating under heavy memory pressure, you might never need to use them.

### There are two main reasons that limit the applicability of in-place operations:

(a) Overwriting values required to compute gradients. This is why variables don’t support `log_`. Its gradient formula requires the original input, and while it is possible to recreate it by computing the inverse operation, it is numerically unstable, and requires additional work that often defeats the purpose of using these functions.

(b) Every in-place operation actually requires the implementation to rewrite the computational graph. Out-of-place versions simply allocate new objects and keep references to the old graph, while in-place operations, require changing the creator of all inputs to the Function representing this operation. This can be tricky, especially if there are many Variables that reference the same storage (e.g. created by indexing or transposing), and in-place functions will actually raise an error if the storage of modified inputs is referenced by any other Variable.
In-place correctness checks

## Second and higher order derivatives

### Computing gradients w.r.t gradients

1. `o = xy + z`
2. `l = o + do_dz`

### Practical application of this in WGAN-GP later in the tutorial

In [29]:
x = torch.randn(5, 3, requires_grad=True)
y = torch.randn(3, 5, requires_grad=True)
z = torch.randn(5, 5, requires_grad=True)

In [30]:
o = torch.mm(x, y) + z ** 2
# if create_graph=False then the resulting gradient is volatile and cannot be used further to compute a second loss.
do_dz = torch.autograd.grad(o, z, grad_outputs=torch.ones(5, 5), retain_graph=True, create_graph=True)
print('do/dz \n\n : %s ' % (do_dz[0]))
l = o ** 3 + do_dz[0]
dl_dz = torch.autograd.grad(l, z, grad_outputs=torch.ones(5, 5))
print('dl/dz \n\n : %s ' % (dl_dz[0]))

do/dz 

 : tensor([[ 1.8460, -0.1780, -0.0409, -3.3022,  0.7370],
        [ 2.4276,  0.2307,  3.4280, -1.6911, -4.2006],
        [ 1.2195,  2.1829,  3.0517,  0.2148,  1.2883],
        [-1.0409,  4.5506, -0.3052, -3.2695,  0.1265],
        [ 1.2656,  0.2457, -3.5047, -0.7261, -3.1899]], grad_fn=<MulBackward0>) 
dl/dz 

 : tensor([[ 3.4253e+01, -9.4891e+00,  1.9251e+00, -5.5240e-01,  3.7409e+01],
        [ 1.9620e+01,  4.2053e+00,  2.2288e+02, -2.8153e+01, -4.8898e+02],
        [ 3.1463e+00,  2.0094e+00,  1.3859e+02,  2.0456e+00,  2.6671e+00],
        [ 1.6217e+00,  9.4163e+02, -1.3887e+01, -4.3227e+02,  6.7477e+00],
        [ 2.7474e+01,  3.1404e+00, -1.5299e+02,  5.1926e-01, -1.3451e+01]]) 
